In [1]:
# Configuration for auto-completion
%config Completer.use_jedi = False
%config Completer.greedy = True

# Enable test mode for CI pipeline
import sys
if len(sys.argv) == 2 and sys.argv[1] == "test":
    test_mode = True
    server = "https://qc-plat-sta.dlr.de/"
else:
    test_mode = False
    server = "https://qc-plat-pro.dlr.de/"


In [ ]:
import time
from qciconnect import Client

# Load environment variables for logging in to QCI Connect
# Insert your personal access token below. (You need to create it on the QCI Connect website on your account page.)
token=%env CI_QCICONNECT_TOKEN

In [3]:
client = Client(token=token, server=server)

In [4]:
client.methods.show()

transpilation_qiskit_compiler


## Compile Job Submission

In [5]:
circuit = """OPENQASM 3.0;
include "stdgates.inc";
qubit[2] q;
h q[0];
cx q[0], q[1];"""

In [6]:
result = client.methods.transpilation_qiskit_compiler({"circuit": circuit})

Job submitted with ID: e4b25832-eab3-4a6a-aec0-5083a6d15301
Job was successful.


In [7]:
print(result.compiled_circuit)

OPENQASM 3.0;
include "stdgates.inc";
qubit[2] q;
h q[0];
cx q[0], q[1];



In [8]:
result = client.methods.transpilation_qiskit_compiler({"circuit": circuit}, {
      "qubit_connectivity": [[0,1]],
      "gate_set": ["h", "x", "cz"],
      "optimization_level": 2
    })

Job submitted with ID: 9715b97c-54bc-4232-9cb4-b4d5cb7523ed
Job was successful.


In [9]:
print(result.compiled_circuit)

OPENQASM 3.0;
include "stdgates.inc";
h $0;
h $1;
cz $0, $1;
h $0;



## Non-blocking compile job submission

In [10]:
promise = client.methods.transpilation_qiskit_compiler({"circuit": circuit}, wait_for_results=False)


Job submitted with ID: 8db67184-0f5c-4e2d-943f-91ed8f41ad6f
Job was successful.


In [11]:
if test_mode:
    print("Waiting in test mode for job to complete...")
    time.sleep(5)
    
result = promise.result

In [12]:
print(result.compiled_circuit)

OPENQASM 3.0;
include "stdgates.inc";
qubit[2] q;
h q[0];
cx q[0], q[1];



## Retrieve result by job ID

In [19]:
result = client.results[promise.job_id]

Job was successful.


In [20]:
print(result.compiled_circuit)

OPENQASM 3.0;
include "stdgates.inc";
qubit[2] q;
h q[0];
cx q[0], q[1];



## QPU Job Submission

In [21]:
client.qpus.show()

Alias                    Qubits  Status
---------------------  --------  -----------
qsea_i_|_digital_twin        10  offline
qsea_i                       10  offline
qsea_ii                      20  coming_soon
xq2i                          8  coming_soon
redac                         8  coming_soon
dinaqc                      100  coming_soon
legato                       20  coming_soon
toccata                      50  coming_soon
xaphiro                      50  coming_soon
sunqc                         8  coming_soon
carina                        8  offline
dedalo                       64  coming_soon
comiqc                       50  coming_soon
xq1i                          4  idle
saxonq_4qb_rt                 3  offline
statevector_simulator        24  running


In [22]:
result = client.qpus.statevector_simulator.submit(circuit, primitive="sampling", shots=10, name="API Demonstration", comment="Submit algorithm X with parameters YZ")

Job submitted with ID: c6882744-1bbf-4df7-9448-470c31e2cce1
Job was successful.


In [23]:
print(result.data)

[[1, 1], [1, 1], [1, 1], [0, 0], [1, 1], [0, 0], [0, 0], [1, 1], [0, 0], [0, 0]]


## Non-blocking QPU job submission

In [24]:
promise = client.qpus.statevector_simulator.submit(circuit, primitive="quantum_state", shots=10, wait_for_results=False, name="API Demonstration", comment="Submit algorithm X with parameters YZ")


Job submitted with ID: 04a15412-8afb-4ec7-b767-60886fa834e7
Job was successful.


In [25]:
if test_mode:
    print("Waiting in test mode for job to complete...")
    time.sleep(5)
    
result = promise.result
print(result.data)

{'[0, 0]': (0.7071067811865476+0j), '[1, 1]': (0.7071067811865475+0j)}


In [ ]:
if not test_mode:
    promise = client.qpus.saxonq_4qb_rt.submit(circuit, "sampling", name="Default circuit on SaxonQ machine", shots=600000, wait_for_results=False)
    

In [ ]:
if not test_mode:
    result = promise.result

In [ ]:
print(result.data)